In [1]:
# Import libraries for data manipulation and ETL pipeline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pandarallel import pandarallel
pandarallel.initialize(verbose=0)
from tqdm import tqdm

In [2]:
# Establish API connection
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = '2a755cb04a18406b9394dbef2f8069dd'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# Connect to SQL Server
import sqlalchemy as sql
from sqlalchemy import text
import pyodbc
server = 'LAPTOP-V3754MEK' 
database = 'Spotify'

engine = sql.create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
conn = engine.connect()
results = engine.execute("SELECT track_uri2 FROM dim_tracks ORDER BY track_uri2")
results = pd.Series(results)
results.head()

0    (spotify:track:0002yNGLtYSYtc0X6ZnFvp)
1    (spotify:track:00039MgrmLoIzSpuYKurn9)
2    (spotify:track:0003Z98F6hUq7XxqSRM87H)
3    (spotify:track:0004ExljAge0P5XWn1LXmW)
4    (spotify:track:0005rgjsSeVLp1cze57jIN)
dtype: object

In [4]:
# Extract tracks from API
output_df = pd.DataFrame()
for t in tqdm(results[190001:200001]):
      output = sp.audio_features(t)
      output_df = output_df.append(pd.DataFrame(output))
output_df.head()

 10%|█         | 1002/10000 [01:36<13:36, 11.03it/s] 

In [ ]:
# Load data into SQL Server
output_df = output_df[['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
output_df = output_df.rename(columns={'key':'music_key', 'mode':'modal'})
output_df.to_sql("audio_features2", engine, if_exists='append', index=False)
print(f'{len(output_df)} rows imported to database.')
new_query = engine.execute('SELECT COUNT(*) FROM audio_features2')
new_query = pd.Series(new_query)
print(f'App contains {new_query[0]} songs.')

10000 rows imported to database.
App contains (190001,) songs.
